In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

traindata=traindata.drop('id',axis=1)
testdata=testdata.drop('id',axis=1)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-1]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-1]

scaler = Normalizer().fit(X)
X = scaler.transform(X)
scaler = Normalizer().fit(x)
x = scaler.transform(x)

Y = np.array(Y)
y = np.array(y)

X = np.reshape(X, (X.shape[0],1,X.shape[1]))
x = np.reshape(x, (x.shape[0],1,x.shape[1]))

Using TensorFlow backend.
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ritika/.virtualen

In [2]:
model = Sequential()
model.add(LSTM(4,input_dim=41))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
a=[]
folds=5

/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(4, input_shape=(None, 41))`
  


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
for i in range(folds):
    print("Training on Fold: ",i+1)
    XTrain, XVal, yTrain, yVal = train_test_split(X, Y, test_size=0.2, random_state = np.random.randint(1,1000, 1)[0])
    model.fit(XTrain, yTrain, nb_epoch=200, validation_data=(XVal, yVal))
    scores = model.evaluate(XVal, yVal, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    a.append(scores[1] * 100)
    print("======="*12, end="\n\n\n")

Training on Fold:  1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 140272 samples, validate on 35069 samples
Epoch 1/200
140272/140272 [==============================] - 19s 134us/step - loss: 0.4500 - accuracy: 0.7464 - val_loss: 0.4091 - val_accuracy: 0.8075
Epoch 2/200
140272/140272 [==============================] - 17s 121us/step - loss: 0.4074 - accuracy: 0.7681 - val_loss: 0.4060 - val_accuracy: 0.7632
Epoch 3/200
140272/140272 [==============================] - 17s 122us/step - loss: 0.4044 - accuracy: 0.7695 - val_loss: 0.4028 - val_accuracy: 0.7679
Epoch 4/200
140272/140272 [==============================] - 17s 122us/step - loss: 0.4026 - accuracy: 0.7685 - val_loss: 0.4021 - val_accuracy: 0.7944
Epoch 5/200
140272/140272 [==============================] - 17s 121us/step - loss: 0.4011 - accuracy: 0.7712 - val_loss: 0.4003 - val_accuracy: 0.7641
Epoch 6/200
140272/140272 [==============================] - 17s 122us/step - loss: 0.3999 - accuracy: 0.7703 - val_loss: 0.3991 - val_accuracy: 0.7592
Epoch 7/200
140272/140272 [==========

140272/140272 [==============================] - 16s 115us/step - loss: 0.3248 - accuracy: 0.8541 - val_loss: 0.3208 - val_accuracy: 0.8554
Epoch 108/200
140272/140272 [==============================] - 16s 117us/step - loss: 0.3250 - accuracy: 0.8540 - val_loss: 0.3221 - val_accuracy: 0.8565
Epoch 109/200
140272/140272 [==============================] - 17s 119us/step - loss: 0.3250 - accuracy: 0.8547 - val_loss: 0.3263 - val_accuracy: 0.8508
Epoch 110/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3248 - accuracy: 0.8542 - val_loss: 0.3235 - val_accuracy: 0.8529
Epoch 111/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.3247 - accuracy: 0.8541 - val_loss: 0.3209 - val_accuracy: 0.8569
Epoch 112/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3243 - accuracy: 0.8546 - val_loss: 0.3217 - val_accuracy: 0.8537
Epoch 113/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.32

140272/140272 [==============================] - 16s 112us/step - loss: 0.3224 - accuracy: 0.8551 - val_loss: 0.3120 - val_accuracy: 0.8622
Epoch 13/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3227 - accuracy: 0.8554 - val_loss: 0.3144 - val_accuracy: 0.8631
Epoch 14/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3228 - accuracy: 0.8550 - val_loss: 0.3121 - val_accuracy: 0.8608
Epoch 15/200
140272/140272 [==============================] - 16s 114us/step - loss: 0.3229 - accuracy: 0.8555 - val_loss: 0.3130 - val_accuracy: 0.8599
Epoch 16/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.3227 - accuracy: 0.8558 - val_loss: 0.3127 - val_accuracy: 0.8617
Epoch 17/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3223 - accuracy: 0.8554 - val_loss: 0.3211 - val_accuracy: 0.8551
Epoch 18/200
140272/140272 [==============================] - 16s 114us/step - loss: 0.3220 - a

...

140272/140272 [==============================] - 16s 112us/step - loss: 0.3182 - accuracy: 0.8568 - val_loss: 0.3083 - val_accuracy: 0.8628
Epoch 130/200
140272/140272 [==============================] - 16s 114us/step - loss: 0.3187 - accuracy: 0.8570 - val_loss: 0.3078 - val_accuracy: 0.8625
Epoch 131/200
140272/140272 [==============================] - 16s 114us/step - loss: 0.3187 - accuracy: 0.8568 - val_loss: 0.3075 - val_accuracy: 0.8630
Epoch 132/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3185 - accuracy: 0.8567 - val_loss: 0.3080 - val_accuracy: 0.8628
Epoch 133/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3183 - accuracy: 0.8570 - val_loss: 0.3105 - val_accuracy: 0.8590
Epoch 134/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3184 - accuracy: 0.8563 - val_loss: 0.3077 - val_accuracy: 0.8629
Epoch 135/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.31

140272/140272 [==============================] - 16s 113us/step - loss: 0.3135 - accuracy: 0.8587 - val_loss: 0.3131 - val_accuracy: 0.8565
Epoch 35/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3130 - accuracy: 0.8587 - val_loss: 0.3132 - val_accuracy: 0.8563
Epoch 36/200
140272/140272 [==============================] - 16s 114us/step - loss: 0.3134 - accuracy: 0.8586 - val_loss: 0.3137 - val_accuracy: 0.8557
Epoch 37/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3128 - accuracy: 0.8590 - val_loss: 0.3163 - val_accuracy: 0.8547
Epoch 38/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3132 - accuracy: 0.8586 - val_loss: 0.3135 - val_accuracy: 0.8558
Epoch 39/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.3134 - accuracy: 0.8585 - val_loss: 0.3161 - val_accuracy: 0.8560
Epoch 40/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3132 - a

140272/140272 [==============================] - 16s 111us/step - loss: 0.3062 - accuracy: 0.8584 - val_loss: 0.3093 - val_accuracy: 0.8564
Epoch 141/200
140272/140272 [==============================] - 16s 114us/step - loss: 0.3063 - accuracy: 0.8584 - val_loss: 0.3046 - val_accuracy: 0.8563
Epoch 142/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.3057 - accuracy: 0.8580 - val_loss: 0.3055 - val_accuracy: 0.8574
Epoch 143/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.3063 - accuracy: 0.8586 - val_loss: 0.3057 - val_accuracy: 0.8569
Epoch 144/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.3054 - accuracy: 0.8587 - val_loss: 0.3048 - val_accuracy: 0.8564
Epoch 145/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.3053 - accuracy: 0.8587 - val_loss: 0.3054 - val_accuracy: 0.8570
Epoch 146/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.30

140272/140272 [==============================] - 16s 113us/step - loss: 0.2994 - accuracy: 0.8581 - val_loss: 0.2967 - val_accuracy: 0.8570
Epoch 46/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.2998 - accuracy: 0.8581 - val_loss: 0.3005 - val_accuracy: 0.8576
Epoch 47/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.2987 - accuracy: 0.8581 - val_loss: 0.2978 - val_accuracy: 0.8570
Epoch 48/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.2991 - accuracy: 0.8579 - val_loss: 0.2962 - val_accuracy: 0.8571
Epoch 49/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.2991 - accuracy: 0.8582 - val_loss: 0.2958 - val_accuracy: 0.8572
Epoch 50/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.2991 - accuracy: 0.8584 - val_loss: 0.2982 - val_accuracy: 0.8573
Epoch 51/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.2992 - a

140272/140272 [==============================] - 16s 113us/step - loss: 0.2968 - accuracy: 0.8578 - val_loss: 0.2952 - val_accuracy: 0.8574
Epoch 152/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.2968 - accuracy: 0.8578 - val_loss: 0.2937 - val_accuracy: 0.8573
Epoch 153/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.2974 - accuracy: 0.8580 - val_loss: 0.2927 - val_accuracy: 0.8575
Epoch 154/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.2970 - accuracy: 0.8584 - val_loss: 0.2926 - val_accuracy: 0.8575
Epoch 155/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.2967 - accuracy: 0.8581 - val_loss: 0.2929 - val_accuracy: 0.8572
Epoch 156/200
140272/140272 [==============================] - 16s 112us/step - loss: 0.2968 - accuracy: 0.8580 - val_loss: 0.2941 - val_accuracy: 0.8567
Epoch 157/200
140272/140272 [==============================] - 16s 113us/step - loss: 0.29

In [4]:
print (a)

[85.81938743591309, 86.30129098892212, 86.28418445587158, 85.50286293029785, 85.7252836227417]


In [4]:
X.shape

(175341, 1, 41)